## This is a testing notebook for the fine-tuned model after unloading peft adaptors onto base model

In [ ]:
!pip install -qqq -U peft trl datasets bitsandbytes

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)

from trl import SFTTrainer


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model name
new_model = "Akil15/finetune_llama_v_0.1"

# Load the entire model on the GPU 0
device_map = {"": 0}

In [7]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [21]:

# Run text generation pipeline with our next model
prompt = input("Question:")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
%time print(result[0]['generated_text'].replace('\\n', '\n').replace('\\', ''))

Question: Write a program implementing sieve of erasthonenes


<s>[INST] Write a program implementing sieve of erasthonenes [/INST]  Sure! Here is a program implementing the Sieve of Eratosthenes in Python:

```
def sieve_of_eratosthenes(n):
    # Create a boolean array of size n+1
    is_prime = [True] * (n + 1)
    
    # Mark composite numbers as false
    for p in range(2, int(n**0.5) + 1):
        if is_prime[p]:
            for i in range(p**0.5, n + 1, p):
                is_prime[i] = False
    
    return [p for p in range(2, n + 1) if is_prime[p]]
```

This program takes an integer `n` as input and returns a list of all prime numbers less than or equal to `n`.

Here's an example usage of the program:
```
print(sieve_of_eratosthenes(30)) # [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
```
I hope this helps! Let me know if you have any questions.
CPU times: user 76 µs, sys: 14 µs, total: 90 µs
Wall time: 95.4 µs


In [28]:

# Run text generation pipeline with our next model
prompt = input("Question:")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
%time print(result[0]['generated_text'].replace('\\n', '\n').replace('\\', ''))

Question: Use a SQL query to extract all the rows from the table \"Employees\" that includes employees with the last name Smith, whose salaries are above the average salary of all employees in the table, and who joined the company before a specific date. Additionally, exclude any employees whose salaries are also above the average salary of all employees with the same job title. \n\nAdditional Instructions:\n1. Calculate the average salary of all employees in the table.\n2. Use the calculated average salary to filter out employees with salaries below the average.\n3. Use the calculated average salary of employees with the same job title to filter out employees whose salaries are above the average for their respective job titles.\n4. Filter out employees who joined the company after a specific date.\n5. Sort the results by employee ID in ascending order.


<s>[INST] Use a SQL query to extract all the rows from the table "Employees" that includes employees with the last name Smith, whose salaries are above the average salary of all employees in the table, and who joined the company before a specific date. Additionally, exclude any employees whose salaries are also above the average salary of all employees with the same job title. 

Additional Instructions:
1. Calculate the average salary of all employees in the table.
2. Use the calculated average salary to filter out employees with salaries below the average.
3. Use the calculated average salary of employees with the same job title to filter out employees whose salaries are above the average for their respective job titles.
4. Filter out employees who joined the company after a specific date.
5. Sort the results by employee ID in ascending order. [/INST]  Sure, here is a SQL query that meets your requirements:
```
SELECT e.*
FROM Employees e
WHERE e.last_name = 'Smith'
AND e.salary > (SE

In [20]:

# Run text generation pipeline with our next model
prompt = input("Question:")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
%time print(result[0]['generated_text'].replace('\\n', '\n').replace('\\', ''))

Question: Write an SQL Query to print a list of all employees who have a higher salary than their managers.


<s>[INST] Write an SQL Query to print a list of all employees who have a higher salary than their managers. [/INST]  Sure! Here is an SQL query to print a list of all employees who have a higher salary than their managers:
```
SELECT * FROM employees
WHERE salary > (SELECT salary FROM manager);
```
This query will select all employees who have a higher salary than their manager. The `>` operator is used to compare the salary of the employee to the salary of the manager.

Note: In this query, `manager` is a table that contains the salary of all managers. If you don't have a `manager` table, you can replace it with the name of the table that contains the salary of all managers.
CPU times: user 97 µs, sys: 0 ns, total: 97 µs
Wall time: 103 µs


In [ ]:
Use a SQL query to extract all the rows from the table \"Employees\" that includes employees with the last name Smith, whose salaries are above the average salary of all employees in the table, and who joined the company before a specific date. Additionally, exclude any employees whose salaries are also above the average salary of all employees with the same job title. \n\nAdditional Instructions:\n1. Calculate the average salary of all employees in the table.\n2. Use the calculated average salary to filter out employees with salaries below the average.\n3. Use the calculated average salary of employees with the same job title to filter out employees whose salaries are above the average for their respective job titles.\n4. Filter out employees who joined the company after a specific date.\n5. Sort the results by employee ID in ascending order.